In [1]:
exec(open("./funcs/tool_funcs.py").read())



In [2]:
def cal_need_update_list_minimax_path(remove_node, X_minimax_path_matrix):
    N = len(X_minimax_path_matrix)
    removal = remove_node
    remaining_list = [i for i in np.arange(N) if i != removal]
    need_update_list = [[] for i in range(N)]
    how_many_counter = np.zeros(N)
    for i in remaining_list:
        for j in remaining_list:
            if i != j:
                if X_minimax_path_matrix[i,j] >= max(X_minimax_path_matrix[i,removal], X_minimax_path_matrix[removal,j]):
                    need_update_list[i].append(j)
                    how_many_counter[i] += 1
    
    how_many = np.sum(how_many_counter > 0)
    
    return need_update_list, how_many

def generate_new_distance_matrix_minimax_path(X_distance_matrix, remove_node):
    X_distance_matrix_new = X_distance_matrix.copy()
 

    N = len(X_distance_matrix)
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_distance_matrix_new[i,j] = np.inf

    X_distance_matrix_new[remove_node,remove_node] = 0  
    
    return X_distance_matrix_new

In [3]:
def dijkstra_one_to_all_minimax_path(distance_matrix, src, need_up_i, remove_node):
    
    N = len(distance_matrix)
    dist = [np.inf] * N
    dist[src] = 0
    visited = [False] * N

    # Priority queue: (distance, node)
    pq = [(0, src)]
    
    kk = len(need_up_i)
    counter = 0
    visited[remove_node] = True

    while pq:
        current_dist, u = heapq.heappop(pq)

        if visited[u]:
            continue
        visited[u] = True
        
        if u in need_up_i:
            counter += 1
            if counter == kk:
                # import pdb;pdb.set_trace()
                return np.array(dist)
 

        # Update distances for neighbors
        for v in range(N):
            if not visited[v] and distance_matrix[u][v] > 0:
                new_dist = max(current_dist, distance_matrix[u][v])
                if new_dist < dist[v]:
                    dist[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))

    return np.array(dist)

In [4]:

def cal_new_minimax_path_matrix_warm(X_distance_matrix, X_minimax_path_matrix, remove_node):
    N = len(X_distance_matrix)
    X_minimax_path_matrix_new = X_minimax_path_matrix.copy()
    remaining_list = [i for i in np.arange(N) if i != remove_node]

    need_update_list, how_many = cal_need_update_list_minimax_path(remove_node, X_minimax_path_matrix)
    
    Cost = np.round(how_many/(N - 1), 3)
    
    print("Cost: ", Cost)

    X_distance_matrix_new = generate_new_distance_matrix_minimax_path(X_distance_matrix, remove_node)
 
    for i in remaining_list:
        if len(need_update_list[i]) > 0:
            temp = dijkstra_one_to_all_minimax_path(X_distance_matrix_new, i, need_update_list[i], remove_node)

            for j in need_update_list[i]:
                X_minimax_path_matrix_new[i,j] = temp[j]
 
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_minimax_path_matrix_new[i,j] = np.inf

    X_minimax_path_matrix_new[remove_node,remove_node] = 0
    
    return X_minimax_path_matrix_new 


In [5]:
def minimax_path_n_to_r_new(distance_matrix, minimax_path_matrix, remaining_list, r, remove_node):
  
    max_jump_list = []
    for t in remaining_list:
        m_jump =  max(distance_matrix[remove_node,t], minimax_path_matrix[t,r])
        max_jump_list.append(m_jump)
    return np.min(max_jump_list)

def minimax_path_r_to_n_new(distance_matrix, minimax_path_matrix, remaining_list, r, remove_node):
    max_jump_list = []
    for t in remaining_list:
        m_jump =  max(minimax_path_matrix[r,t], distance_matrix[t, remove_node]) 
        max_jump_list.append(m_jump)
    return np.min(max_jump_list)

     
def cal_n_minimax_path_new(distance_matrix, minimax_path_matrix, remove_node, remaining_list):
    for r in remaining_list:
        minimax_path_matrix[remove_node, r] = minimax_path_n_to_r_new(distance_matrix, minimax_path_matrix, remaining_list, r, remove_node)
        minimax_path_matrix[r, remove_node] = minimax_path_r_to_n_new(distance_matrix, minimax_path_matrix, remaining_list, r, remove_node)
        
    for i in remaining_list:        
        for j in remaining_list:
            if i < j:
                minimax_path_matrix[i,j] =  update_minimax_path_ij_new(distance_matrix, minimax_path_matrix, remove_node, i, j)
                minimax_path_matrix[j,i] =  update_minimax_path_ij_new(distance_matrix, minimax_path_matrix, remove_node, j, i)
                
def update_minimax_path_ij_new(distance_matrix, minimax_path_matrix, remove_node, i,j):
    m1 = minimax_path_matrix[i,j]
    m2 =  max(minimax_path_matrix[i,remove_node], minimax_path_matrix[remove_node,j])
    return np.min((m1,m2))



In [6]:
def cal_which_to_remove(X_minimax_path_matrix, edge_nodes):
   
    _, how_many0 = cal_need_update_list_minimax_path(edge_nodes[0], X_minimax_path_matrix)
    _, how_many1 = cal_need_update_list_minimax_path(edge_nodes[1], X_minimax_path_matrix)
    if how_many0 < how_many1:
        remove_node = edge_nodes[0]
    else:
        remove_node = edge_nodes[1]
    # print(how_many0, how_many1, edge_nodes, remove_node)
    return remove_node
    
    
    

In [7]:

def cal_minimax_path_matrix_after_graph_update(X_distance_matrix, X_minimax_path_matrix, edge_nodes, X_distance_matrix_updated):
    
    N = len(X_distance_matrix)
    
    remove_node = cal_which_to_remove(X_minimax_path_matrix, edge_nodes)

    X_minimax_path_matrix_updated = cal_new_minimax_path_matrix_warm(X_distance_matrix, X_minimax_path_matrix, remove_node)
 
    remaining_list = [i for i in np.arange(N) if i != remove_node]
    
    cal_n_minimax_path_new(X_distance_matrix_updated, X_minimax_path_matrix_updated, remove_node, remaining_list)
    
    return X_minimax_path_matrix_updated

 
    

In [8]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)

In [9]:
X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")
 

In [10]:

edge_nodes = [3, 9]

edge_weight = 50    

X_distance_matrix_new = X_distance_matrix.copy()
X_distance_matrix_new[edge_nodes[0],edge_nodes[1]] = edge_weight 

X_minimax_path_matrix = floyd_warshall_minimax_path(X_distance_matrix)

In [11]:
start = time.time()
X_minimax_path_matrix_new_warm = cal_minimax_path_matrix_after_graph_update(X_distance_matrix, X_minimax_path_matrix, edge_nodes, X_distance_matrix_new)
end = time.time()


time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

Cost:  1.0


In [12]:
print(f"Time used: {time_used1_warm}s" )


Time used: 0.508s


In [13]:
start = time.time()
X_minimax_path_matrix_new_floyd_warshall = floyd_warshall_minimax_path(X_distance_matrix_new)
end = time.time()


time_used2_floyd = end - start
time_used2_floyd = np.round(time_used2_floyd, 3)

In [14]:
print(f"Time used: {time_used2_floyd}s" )


Time used: 1.251s


In [15]:
print(np.allclose(X_minimax_path_matrix_new_warm, X_minimax_path_matrix_new_floyd_warshall))


True


In [16]:
print(f"Ratio: {np.round(time_used1_warm/time_used2_floyd, 2)}")


Ratio: 0.41


In [17]:
X_minimax_path_matrix_new_warm[1]


array([11.,  0., 15.,  8., 24., 18., 18.,  8., 12., 17.,  8., 17.,  8.,
        8., 11.,  8., 17., 17.,  9., 11., 10., 15., 11., 15.,  9.,  9.,
       15., 17., 21.,  8., 14., 13., 17., 11.,  8., 14., 29.,  8., 18.,
       39.,  8.,  9.,  8., 21., 18., 14.,  8., 47., 10., 16., 17.,  8.,
        8., 18., 21.,  8., 13., 21., 18., 10.,  8.,  8., 19.,  8., 42.,
       17., 21., 14., 14.,  8., 15., 15., 13.,  8., 10.,  8.,  8., 17.,
       18., 24., 20., 28., 10., 10., 18.,  8., 18., 27., 10., 22., 15.,
       17., 26.,  8.,  9., 11., 18., 13.,  8., 11.])